In [1]:
import os
#import keras
import torch
import numpy as np
from collect import s3_download_file
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader



In [ ]:
s3_download_file("kagglecatsanddogs_5340.zip", "data.zip")

In [ ]:
!unzip -q -n data.zip

In [2]:
num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = b"JFIF" in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print(f"Deleted {num_skipped} corrupted images.")

Deleted 0 corrupted images.


In [3]:
# Define the directory path to your dataset
data_dir = 'PetImages'

# Define transformations to be applied to each image
transform = transforms.Compose([
    transforms.Resize((50, 50)),  # Resize the images to a fixed size
    transforms.ToTensor(),          # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the images
])

# Create the dataset
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Split the dataset into training and validation sets (optional)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

# Check the dataset class labels
print(dataset.classes)  # Output: ['Cat', 'Dog']

# Iterate through the data loader
for images, labels in train_loader:
    print(images.size(), labels.size())  # Example output: torch.Size([32, 3, 224, 224]) torch.Size([32])
    break

['Cat', 'Dog']
torch.Size([32, 3, 50, 50]) torch.Size([32])


In [5]:
# Define the model (use a pre-trained ResNet and modify the final layer)
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 2 classes: Cat and Dog

# Move the model to the GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f'Epoch {epoch+1}/{num_epochs}, \n Training Loss: {epoch_loss:.4f}')

    # Validation loop
    model.eval()
    val_loss = 0.0
    correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            correct += torch.sum(preds == labels.data)

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct.double() / len(val_loader.dataset)
    print(f' Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}')


Epoch 1/10, 
 Training Loss: 0.4139
Validation Loss: 0.4070, Accuracy: 0.7826
Epoch 2/10, 
 Training Loss: 0.2825
Validation Loss: 0.3051, Accuracy: 0.8706
Epoch 3/10, 
 Training Loss: 0.2249
Validation Loss: 0.3125, Accuracy: 0.8704
Epoch 4/10, 
 Training Loss: 0.1852
Validation Loss: 0.3332, Accuracy: 0.8610
Epoch 5/10, 
 Training Loss: 0.1535
Validation Loss: 0.3493, Accuracy: 0.8748
Epoch 6/10, 
 Training Loss: 0.1294
Validation Loss: 0.2810, Accuracy: 0.8881
Epoch 7/10, 
 Training Loss: 0.0934
Validation Loss: 0.3358, Accuracy: 0.8913
Epoch 8/10, 
 Training Loss: 0.0891
Validation Loss: 0.4179, Accuracy: 0.8648
Epoch 9/10, 
 Training Loss: 0.0676
Validation Loss: 0.3852, Accuracy: 0.8746
Epoch 10/10, 
 Training Loss: 0.0659
Validation Loss: 0.3709, Accuracy: 0.8830


In [19]:
# Test with new image
from PIL import Image

image = Image.open("PetImages/Cat/0.jpg").convert('RGB')
# Put the image in a batch of size 1
image = transform(image).unsqueeze(0)
print(image.shape)
image = image.to(device)
 # Make prediction
with torch.no_grad():
    outputs = model(image)
    _, preds = torch.max(outputs, 1)

# Map the prediction to the class label
class_names = ['Cat', 'Dog']
predicted_class = class_names[preds.item()]
print(predicted_class)

AttributeError: 'Image' object has no attribute 'shape'

In [30]:
#first_batch_example = next(iter(val_loader))[0].to(device)
first_batch_example = torch.randn(1, 3, 50, 50).to(device)
torch.onnx.export(model, first_batch_example, "cats-vs-dogs.onnx")